In [1]:
import pandas as pd       
import seaborn as sns    
import matplotlib.pyplot as plt  
import numpy as np        


In [8]:
train=pd.read_csv(r"C:\Users\jojan\OneDrive\文档\CERTS and IMPs\A my JOURNEY\INTERNSHIPS\CODSOFT\Projects\SMS Spam classification\spam.csv",encoding='latin')
train.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [10]:
train.drop(train.columns[2:],axis=1,inplace=True)
train.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = train['v2']
y = train['v1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train) # ham=0, spam=1
y_test=le.transform(y_test)

# 4. Build pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000)),
    ('clf', MultinomialNB())
])

# 5. Define hyperparameter grid
param_grid = {
    'clf__alpha': [0.1, 0.5, 1.0]  # Smoothing parameter
}
# 6. Cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 7. Grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 8. Evaluate best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_train_pred=best_model.predict(X_train)
print("Best Parameters:", grid_search.best_params_)
print("Train Accuracy:", accuracy_score(y_train,y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'clf__alpha': 0.5}
Train Accuracy: 0.9921471842046219
Test Accuracy: 0.97847533632287

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.99      0.85      0.91       149

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Confusion Matrix:
 [[965   1]
 [ 23 126]]
